In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Merge with OPEN AI SCORE and DATAFRAME

In [ ]:
df_final=pd.read_csv('/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/df_cleaned_final_01.csv')
df_scores_gpt4=pd.read_excel('/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/Scored_Articles_3000.xlsx')


In [ ]:
#check duplicates for manual.summary in df_final

df_final['Manual.summary'].duplicated().sum()

In [ ]:
print()

In [ ]:
df_final.columns

In [ ]:
print(df_scores_gpt4.columns)
print(df_scores_gpt4.shape[0])

In [ ]:
#There are matches between the two dataframes that exceed, it is because of the duplicates that we decide to kept if they were in different plataforms

# Check the number of matches
matches = df_final['Manual.summary'].isin(df_scores_gpt4['Manual.summary'])
number_of_matches = matches.sum()

print(f"Number of matching entries: {number_of_matches}")

In [ ]:
#  From what I am understanding these are things that are non matched in our dataframe, so they were dropped before the final dataframe, but they were still in the random ones. 
#  Check the number of matches
matches = df_scores_gpt4['Manual.summary'].isin(df_final['Manual.summary'])
number_of_matches = matches.sum()

print(f"Number of matching entries: {number_of_matches}")

In [ ]:
pd.set_option('display.width', 1000)  # Set the display width to 1000 characters wide
pd.set_option('display.max_colwidth', None)  # Remove truncation of column content


# Check for non-matching entries
non_matches = ~df_scores_gpt4['Manual.summary'].isin(df_final['Manual.summary'])
non_matching_entries = df_scores_gpt4[non_matches]

# Print the number of non-matching entries
print(f"Number of non-matching entries: {non_matching_entries.shape[0]}")

# Display the non-matching entries
if non_matching_entries.shape[0] > 0:
    print("Non-matching entries in 'Manual.summary' from df_scores_gpt4:")
    print(non_matching_entries['Manual.summary'])
else:
    print("No non-matching entries found.")

In [ ]:
#Reset options every option
pd.reset_option('all')

# First, check the number of matches before merging
matches = df_final['Manual.summary'].isin(df_scores_gpt4['Manual.summary'])
number_of_matches_before_merge = matches.sum()
print(f"Number of matches before merge: {number_of_matches_before_merge}")

# Perform the merge to add the 'OpenAI_Score' from df_scores_gpt4 to df_final
# Make sure there's no column name conflict by specifying suffixes if needed
df_final = df_final.merge(df_scores_gpt4[['Manual.summary', 'OpenAI_Score']],
                          on='Manual.summary', how='left', suffixes=('', '_drop'))

# After merge, drop any duplicate or unnecessary columns that may have resulted from the merge
df_final.drop([col for col in df_final.columns if 'drop' in col], axis=1, inplace=True)

# Print the result of the merge to verify
df_final.head(4)

In [ ]:
# Print the total number of values in the 'OpenAI_Score' column
total_values = len(df_final['OpenAI_Score'])
print(f"Total number of values in 'OpenAI_Score': {total_values}")

# Print the number of non-null values in the 'OpenAI_Score' column
non_null_values = df_final['OpenAI_Score'].count()
print(f"Number of non-null values in 'OpenAI_Score': {non_null_values}")

# Seeing the score based on the Open_AI Scores

In [ ]:
# Keep columns only that are non-null in OPENAI_SCORE
df_open_ai_scores= df_final[df_final['OpenAI_Score'].notnull()]
df_open_ai_scores.to_csv('/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/df_open_ai_scores.csv')

# Train of the final model


In [ ]:
df_final.columns

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [ ]:
# # Load the fine-tuned model and tokenizer
model_path = "/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/finetuned-FOMC-RoBERTa"
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(model_path)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a pipeline for sentiment analysis
classifier = pipeline("sentiment-analysis", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Select only the first 100 rows for processing
unlabeled_texts = df_final['Manual.summary'].head(1000).tolist()

# Make predictions
predictions = classifier(unlabeled_texts)

# Add predictions to the dataframe for the first 100 rows
df_final.loc[:999, 'bert_predictions_test'] = [pred['label'] for pred in predictions]  # using loc to ensure we only modify the first 100 rows

# Display the updated part of the dataframe
print(df_final.head(1000))


In [ ]:
df_final

In [ ]:
unique_counts = df_final['bert_predictions_test'].value_counts(dropna=True)
print(unique_counts)


In [ ]:
unique_counts = df_final['bert_predictions_test'].value_counts(dropna=True)
print(unique_counts)


In [ ]:
pd.set_option('display.width', 1000)  # Set the display width to 1000 characters wide
pd.set_option('display.max_colwidth', None)  # Remove truncation of column content

#Print examples of Manual.Summary Label_1 predictions
print(df_final['Manual.summary'][df_final['bert_predictions_test'] == 'LABEL_2'].head(10))

In [ ]:
print(df_final['Manual.summary'][df_final['bert_predictions_test'] == 'LABEL_1'].head(10))

In [ ]:
print(df_final['Manual.summary'][df_final['bert_predictions_test'] == 'LABEL_0'].head(10))

# Test to solve the issue from the previous run 

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load the fine-tuned model and tokenizer
model_path = "/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/finetuned-FOMC-RoBERTa"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, truncation=True, padding="max_length", max_length=512)

# Setup the pipeline for sentiment analysis using the loaded model and tokenizer
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# DataFrame column to list
summaries = df_final['Manual.summary'].tolist()

# Error indices identified from previous runs
error_indices = [1785, 7148, 7780]
ranges_to_process = []

# Calculate ranges: 10 before, the index itself, and 10 after
for index in error_indices:
    start = max(0, index - 10)
    end = min(len(summaries), index + 11)
    ranges_to_process.append(range(start, end))

# Process the data in these specific ranges
predictions = [None] * len(summaries)  # Initialize a list of None to store predictions
for range_to_process in ranges_to_process:
    for i in range_to_process:
        try:
            # Make predictions for the current summary
            batch_predictions = classifier([summaries[i]])
            predictions[i] = batch_predictions[0]['label']
        except Exception as e:
            print(f"An error occurred while processing summary at index {i}: {e}")
            predictions[i] = 'Error'  # Mark this as an error

# Safely add predictions to the dataframe ensuring all are accounted for
df_final['bert_predictions_everything'] = predictions

# Display part of the updated dataframe
print(df_final.head())


In [ ]:
unique_counts = df_final['bert_predictions_everything'].value_counts(dropna=True)
print(unique_counts)

# Applying this for the full model

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load the fine-tuned model and tokenizer
model_path = "/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/finetuned-FOMC-RoBERTa"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Setup the pipeline for sentiment analysis using the loaded model and tokenizer
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# DataFrame column to list
summaries = df_final['Manual.summary'].tolist()

# Initialize a predictions list
predictions = []

# Process each summary individually
for i, summary in enumerate(summaries):
    try:
        # Make predictions for the current summary
        batch_predictions = classifier([summary])  # Processing one summary at a time
        predictions.extend(batch_predictions)
    except Exception as e:
        print(f"An error occurred while processing summary at index {i}: {e}")
        predictions.append({'label': 'Error'})  # Mark this as an error

# Add predictions to the dataframe
df_final['bert_predictions_everything'] = [pred['label'] for pred in predictions]

# Display part of the updated dataframe
print(df_final.head())


In [ ]:
unique_counts = df_final['bert_predictions_everything'].value_counts(dropna=True)
print(unique_counts)

In [ ]:
# Total counts for each label
label_1_count = 23722
label_0_count = 5224
label_2_count = 1211

# Total counts combined
total_count = label_1_count + label_0_count + label_2_count

# Calculating percentages for each label
percentage_label_1 = (label_1_count / total_count) * 100
percentage_label_0 = (label_0_count / total_count) * 100
percentage_label_2 = (label_2_count / total_count) * 100

percentage_label_1, percentage_label_0, percentage_label_2

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv('/Users/ruimaciel/Desktop/Local_ECB_Cacophony_Master_Thesis/df_final_with_bert_predictions.csv')